# 特征工程

## 特征选择
特征选择是特征工程的第一步，它关系到我们机器学习算法的上限。因此原则是**尽量不错过一个可能有用的特征**，但是也**不滥用太多的特征**。
### 过滤法
#### 方差过滤
  指定一个方差的阈值，当方差小于这个阈值的特征会被我们筛掉。sklearn中的VarianceThreshold类可实现这一操作
```
import numpy as np
from sklearn.feature_selection import VarianceThreshold
data = np.array([[1,2,3],[1,1,3],[4,5,6]])
vt = VarianceThreshold(threshold=2)
vt.fit_transform(data)
```
#### 相关系数
分别计算所有训练集中各个特征与输出值之间的相关系数，设定一个阈值，选择相关系数较大的部分特征。 
```
from sklearn.feature_selection import SelectKBest
import numpy as np
X = np.array([[1,2,3],[1,1,3],[4,5,6]])
y = np.array([1,2,1])
X_corr = SelectKBest(lambda X, y: np.array(list(
    map(lambda x: np.corrcoef(x.T, y.T)[0,1], X.T))),k=2).fit_transform(X, y)
```
(注： `np.corrcoef`返回的是矩阵)
#### 假设检验
_卡方检验_ 可以检验某个特征分布和输出值分布之间的相关性。在sklearn中，可以使用chi2这个类来做卡方检验得到所有特征的卡方值与显著性水平P临界值，我们可以给定卡方值阈值， 选择卡方值较大的部分特征。卡方检验原理可参见[此文](https://segmentfault.com/a/1190000003719712)。
```
from sklearn.feature_selection import SelectKBest ,chi2
X = np.array([[1,2,3],[1,1,3],[4,5,6]])
y = [1,2,1]
X_chi2 = SelectKBest(chi2, k=2).fit_transform(X, y)
```

除此之外还有 _f-检验_ 及 _t-检验_
#### 互信息
互信息值越大，说明该特征和输出值之间的相关性越大，越需要保留。在sklearn中，可以使用mutual_info_classif(分类)和mutual_info_regression(回归)来计算各个输入特征和输出值之间的互信息。关于互信息详细介绍可查看机器学习决策树相关理论。
```
from sklearn.feature_selection import SelectKBest
from minepy import MINE
def mic(x, y):
    m = MINE()
    m.compute_score(x, y) 
    return m.mic()
X_mutal = SelectKBest(lambda X, Y: np.array(list(
    map(lambda x:mic(x, Y), X.T))), k=2).fit_transform(X, y)
```
  
以上就是过滤法的主要方法，在没有思路时可选取卡方检验及互信息。
### 包装法
最常用的包装法是递归消除特征法(recursive feature elimination,以下简称RFE)。递归消除特征法使用一个机器学习模型来进行多轮训练，每轮训练后，消除若干权值系数的对应的特征，再基于新的特征集进行下一轮训练。在sklearn中，可以使用RFE函数来选择特征。
### 嵌入法选择特征
嵌入法也是用机器学习的方法来选择特征，但是它和RFE的区别是它不是通过不停的筛掉特征来进行训练，而是使用的都是特征全集。在sklearn中，使用SelectFromModel函数来选择特征。最常用的是使用L1正则化和L2正则化来选择特征。  
最常用的是使用L1正则化和L2正则化来选择特征。在正则化模型中，正则化惩罚项越大，那么模型的系数就会越小。当正则化惩罚项大到一定的程度的时候，部分特征系数会变成0，当正则化惩罚项继续增大到一定程度时，所有的特征系数都会趋于0. 但是我们会发现一部分特征系数会更容易先变成0，这部分系数就是可以筛掉的。也就是说，我们选择特征系数较大的特征。常用的L1正则化和L2正则化来选择特征的基学习器是逻辑回归。
### 寻找高级特征
在我们拿到已有的特征后，我们还可以根据需要寻找到更多的高级特征。比如有车的路程特征和时间间隔特征，我们就可以得到车的平均速度这个二级特征。根据车的速度特征，我们就可以得到车的加速度这个三级特征，根据车的加速度特征，我们就可以得到车的加加速度这个四级特征。。。也就是说，高级特征可以一直寻找下去。  
在Kaggle之类的算法竞赛中，高分团队主要使用的方法除了**集成学习算法**，剩下的主要就是**在高级特征上面做文章**。所以寻找高级特征是模型优化的必要步骤之一。  
寻找高级特征最常用的方法有：
1. 若干项特征加和：我们假设你希望根据每日销售额得到一周销售额的特征。你可以将最近的7天的销售额相加得到。
2. 若干项特征之差：假设你已经拥有每周销售额以及每月销售额两项特征，可以求一周前一月内的销售额。
3. 若干项特征乘积：假设你有商品价格和商品销量的特征，那么就可以得到销售额的特征。
4. 若干项特征除商：假设你有每个用户的销售额和购买的商品件数，那么就是得到该用户平均每件商品的销售额。

## 特征表达
### 缺失值处理


In [34]:
from sklearn.feature_selection import SelectKBest
from minepy import MINE
def mic(x, y):
    m = MINE()
    m.compute_score(x, y) 
    return m.mic()
X_mutal = SelectKBest(lambda X, Y: np.array(list(
    map(lambda x:mic(x, Y), X.T))), k=2).fit_transform(X, y)

In [29]:
from sklearn.feature_selection import SelectKBest ,chi2
chi2(X, y)

(array([0.75  , 1.5625, 0.375 ]), array([0.38647623, 0.21129955, 0.54029137]))

In [5]:
y

NameError: name 'y' is not defined

啊